# Code to generate Frustration Fouls and Euro Fouls

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "./figs/"

league = "nba"
season_yr = "2022"
season_type = "Regular Season"
games_id = pbp_season(league="nba",season_yr="2023",season_type="Regular Season")
games_list = pbp_games(games_id)

In [3]:
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
pos_store = []
TO_Miss_pID = []
Foul_pID = []
Foul_tID = []
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= 5:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                # print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                # print ("Foul Player: {0}".format(possession_event.player1_id))

100%|██████████| 89/89 [00:00<00:00, 3405.68it/s]


In [4]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

In [5]:
Player_Name = []
a = Foul_pID
Foul_pID1, Fouls = np.unique(a, return_counts=True)  

for pID in Foul_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [6]:

for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(Foul_pID1))

126
126


In [7]:
data_euroP = pd.DataFrame({'Foul_pID':Foul_pID1,'Player_Name':Player_Name,'Fouls':Fouls})
data_euroP = data_euroP[data_euroP['Player_Name'] != 'abc']
# data_euroP.to_csv ('Fast_Fouls_20_21_updated.csv',index=False)

In [8]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

In [9]:
np.sum(Fouls)

172

In [10]:
data_euroT = pd.DataFrame({'Foul_tID':Foul_tID1,'Team_Name':Team_Name,'Fouls':Fouls})
# data_euroT.to_csv ('Fast_Fouls_20_21_teams_updated.csv',index=False)
data_euroT

,Foul_tID,Team_Name,Fouls
0,1610612737,Atlanta Hawks,7
1,1610612738,Boston Celtics,5
2,1610612739,Cleveland Cavaliers,9
3,1610612740,New Orleans Pelicans,2
4,1610612741,Chicago Bulls,6
5,1610612742,Dallas Mavericks,4
6,1610612743,Denver Nuggets,6
7,1610612744,Golden State Warriors,6
8,1610612745,Houston Rockets,7
9,1610612746,Los Angeles Clippers,7


In [11]:
FFoul_pID =[]
k = 0
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])
        k +=1
        print(k)
        clear_output(wait=True)

45


In [12]:
Player_Name = []
a = FFoul_pID
FFoul_pID, Fouls = np.unique(a, return_counts=True)
for pID in FFoul_pID:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [13]:
for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(FFoul_pID))

38
38


In [14]:
data_frust = pd.DataFrame({'FFoul_pID':FFoul_pID,'Player':Player_Name,'Fouls':Fouls})
data_frust = data_frust[data_frust['Player'] != 'abc']
# data_frust.to_csv ('Frustration_Fouls_20_21_updated.csv',index=False)

In [15]:
data_frust['rank'] = data_frust['Fouls'].rank(ascending=False) 
data_frust = data_frust.sort_values(by=['rank'])
data_frust_plot = data_frust.drop(columns=['FFoul_pID', 'rank'])
data_frust_plot.reset_index(drop=True,inplace=True)
data_frust_plot.reset_index(drop=False,inplace=True)
data_frust_plot["#"] = data_frust_plot["index"] +1
data_frust_plot = data_frust_plot[["#","Player","Fouls"]]
data_frust_plot = data_frust_plot.head(10)
data_frust_plot

,#,Player,Fouls
0,1,Russell Westbrook,3
1,2,Tobias Harris,2
2,3,Jordan Clarkson,2
3,4,Marcus Smart,2
4,5,D'Angelo Russell,2
5,6,Charles Bassey,1
6,7,Quentin Grimes,1
7,8,Talen Horton-Tucker,1
8,9,Ty Jerome,1
9,10,Anthony Edwards,1


In [16]:
def plot_table(df,var,sort="Player",title=" ",col_width=15):
    fig = go.Figure(data=[go.Table(
        columnwidth=[5,40,col_width],
        header=dict(values=list(df.columns),
                    fill_color='blue',
                    align=['center','left','center'],
                    font=dict(color='snow',family="Arial Black, monospace", size=12),
                    line_color="snow"
                    ),
        cells=dict(values=[df["#"],df[sort],df[var]],
                fill_color='lavender',
                align=['center','left','center'],
                height=23,
                line_color="grey",#lightgrey",
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
    fig.update_layout(title=dict(text=title,y=0.98,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
    tab_width = 250 + col_width
    tab_height = 310
    fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
    fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
    fig.add_annotation(x=1.0, y=0.0,text="Source: pbpstats",showarrow=False,xshift=1,yshift=5)
    # fig.update_layout(autosize=True)
    fig.show()
    # fig.write_image(fig_DIR+f"{var}.png", scale=2)
    return fig

In [17]:
fig = plot_table(data_frust_plot,"Fouls",title="Frustration Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)

In [18]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

data_euro = pd.DataFrame({'Foul_tID':Foul_tID1,'Team':Team_Name,'Fouls':Fouls})
data_euro['rank'] = data_euro['Fouls'].rank(ascending=False) 
data_euro = data_euro.sort_values(by=['rank'])
data_euro_plot = data_euro.drop(columns=['Foul_tID', 'rank'])
data_euro_plot.reset_index(drop=True,inplace=True)
data_euro_plot.reset_index(drop=False,inplace=True)
data_euro_plot["#"] = data_euro_plot["index"] +1
data_euro_plot = data_euro_plot[["#","Team","Fouls"]]
data_euro_plot = data_euro_plot.head(10)
data_euro_plot

,#,Team,Fouls
0,1,Detroit Pistons,14
1,2,Utah Jazz,11
2,3,Cleveland Cavaliers,9
3,4,Brooklyn Nets,8
4,5,Portland Trail Blazers,7
5,6,Indiana Pacers,7
6,7,Orlando Magic,7
7,8,Los Angeles Lakers,7
8,9,Los Angeles Clippers,7
9,10,Atlanta Hawks,7


In [19]:
fig = plot_table(data_euro_plot,"Fouls",sort="Team",title="Euro Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)